In [45]:
!pip install langdetect
!pip install timeout_decorator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for timeout_decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5005 sha256=ceeccc2f525adf9d41174e32168e5dbfb0ce9f9dd01b9b64ba308ff59677d0b0
  Stored in directory: /root/.cache/pip/wheels/68/2f/bc/76f1192d474666d41ae6f09813fccbd00fe3f07e8261c4cff5
Successfully built timeout_decorator


In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from urllib.parse import urlparse
import pandas as pd
import os
import json
from src.Web_Scraping.basic_functions import (is_scraping_allowed, remove_cookie_banners, getWebsiteText, getWebsiteText_v2, format_string, remove_sentence_with_keyword)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# path to csv file
data = pd.read_csv("/content/drive/MyDrive/Startup_Scraping/startup_list_0_5000_prep.csv", index_col=0)

In [7]:
data.head()

,name,state,linkedin_industry,website_url,check_robots,original_idx
0,itravel,Nordrhein-Westfalen,Tourism,https://www.itravel.de/,True,0
1,ZOONO GERMANY,Nordrhein-Westfalen,Life Science & Healthcare,https://www.zoonodeutschland.com/,False,1
2,kunveno,NaN,IT and Communications,https://kunveno.de,True,2
3,t2k: Text to Knowledge,Sachsen,IT and Communications,https://text2knowledge.de,True,3
4,Scopas,Berlin,IT and Communications,https://www.scopas.io/,True,4


Check the robots.txt of each web page, by adjusting range(1751, 3500)
-> all data: len(data)

In [ ]:
for i in range(1751, 3500):
  print(i) # only progress information
  url = data['website_url'][i]
  result = is_scraping_allowed(url)
  data.loc[i, "check_robots"] = result

In [ ]:
#Save data as new csv
data.to_csv("/content/drive/MyDrive/")

In [ ]:
#Counts True and False values in column "check_robots"
data["check_robots"].value_counts()

check_robots
True     990
False    509
Name: count, dtype: int64

Gets Text from the allowed websites and saves it as txt

In [ ]:
#path for the text files

path = "./txt_files_3501_5000/"
os.makedirs(path, exist_ok=True)

In [ ]:
for i in range(0, 1501):
  if(data["check_robots"][i]==True):
    url = data['website_url'][i]
    text = getWebsiteText(url)
    lines = text.splitlines()
    new_lines = [line for line in lines if line]
    new_text = "\n".join(new_lines)
    # lowercase + remove everything in name but chars
    # because some names contains symbols e.g. ":" -> would destroy a textfile
    name = data["name"][i].lower()
    name = re.sub(r"[^a-z]", "", name)
    print(f"{i}: {name}") # only progress information
    with open(path+name+".txt", "w", encoding="utf-8") as f:
      f.write(new_text)


Gets Text from the allowed websites and saves as one json file

In [65]:
data = pd.read_csv("/content/drive/MyDrive/Startup_Scraping/startup_list_0_5000_prep.csv", index_col=0)
data['original_idx'] = data.index
data = data.loc[data["check_robots"] == True]
#data.drop(671, inplace=True) # instead of drop set to False
#data.reset_index(drop=True, inplace=True)

In [67]:
data_dict = {}
# Iterate over the rows of the DataFrame
for index, row in data.iterrows():
    print(index)
    name = row['name']
    id = row['original_idx']
    url = row['website_url']

    try:
      #text = getWebsiteText(url)

      text = getWebsiteText_v2(url)
    except timeout_decorator.TimeoutError:
        # Skip this iteration if the function execution exceeds 10 seconds
        text = "timeout-skip"
        continue

    # Remove
    lines = text.splitlines()
    new_lines = [line for line in lines if line]
    new_text = "\n".join(new_lines)
    website_text = format_string(new_text)


    # Create a dictionary for the current row and add it to the data_dict
    row_dict = {
        'name': name,
        'original_idx': id,
        'website_url': url,
        'website_text': website_text
    }
    data_dict[index] = row_dict

# Print the resulting dictionary
#for index, row_dict in data_dict.items():
#    print(f"Row {index}: {row_dict}")


json_object = json.dumps(data_dict, indent=4, ensure_ascii=False)
with open("/content/drive/MyDrive/Startup_Scraping/startup_list_0_5000_without_cookie.json", "w", encoding='utf-8') as outfile:
    outfile.write(json_object)

0
2
3
4
5
8
9
10
11
12
14
18
19
20
22
25
26
28
30
31
33
36
37
38
43
44
46
48
50
53
56
57
59
60
62
63
66
68
69
70
72
73
76
79
80
81
82
83
85
86
89
91
92
93
94
97
98
99
101
103
104
105
106
107
108
110
112
113
115
117
120
121
123
125
126
127
128
129
130
131
132
136
137
138
140
141
143
146
147
148
149
150
151
152
153
154
155
157
158
161
165
167
171
174
175
176
177
178
179
181
182
183
185
188
190
192
195
196
197
198
202
203
205
206
207
208
210
211
212
214
215
217
219
223
225
226
229
230
231
232
233
235
238
239
240
243
244
246
247
252
253
255
256
258
259
260
261
265
266
267
268
270
272
274
275
276
279
280
282
285
289
291
292
293
295
297
299
300
301
302
303
304
306
307
310
311
314
315
316
317
318
319
320
321
322
323
325
326
327
328
329
330
331
332
333
334
336
337
339
340
343
347
350
352
353
354
356
358
359
361
364
369
370
371
372
373
377
378
380
381
382
383
384
386
389
391
392
394
395
398
399
402
403
405
406
407
408
409
410
411
412
413
414
416
417
419
420
421
423
425
427
428
429
430
431
433
4

1716
1717
1719
1720
1721
1722
1723
1728
1731
1732
1733
1735
1737
1739
1740
1741
1742
1746
1747
1748
1750
1752
1754
1755
1756
1757
1758
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1775
1776
1778
1780
1781
1782
1785
1787
1788
1789
1790
1792
1794
1797
1800
1801
1802
1803
1804
1805
1811
1812
1813
1814
1815
1816
1818
1820
1821
1824
1825
1828
1831
1832
1833
1834
1835
1836
1837
1838
1839
1841
1843
1845
1846
1847
1848
1850
1852
1854
1855
1856
1857
1860
1861
1862
1864
1866
1870
1871
1872
1874
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1888
1889
1891
1893
1894
1895
1896
1898
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1911
1912
1913
1914
1915
1916
1919
1920
1921
1922
1923
1924
1926
1927
1928
1929
1930
1932
1934
1935
1938
1939
1940
1941
1942
1943
1944
1945
1946
1948
1950
1951
1952
1953
1954
1955
1957
1960
1962
1963
1964
1965
1967
1968
1969
1971
1979
1980
1981
1983
1984
1985
1986
1987
1989
1992
1994
1995
1996
1997
1998
2002
2005
2007
2008
2010
2013
2014
2016
2017
2018


3994
3995
3996
3999
4002
4003
4004
4005
4006
4008
4010
4011
4012
4013
4016
4017
4018
4022
4023
4024
4028
4029
4031
4032
4033
4034
4035
4036
4037
4038
4040
4041
4042
4045
4048
4049
4050
4053
4056
4057
4058
4060
4061
4062
4063
4064
4065
4066
4067
4069
4070
4071
4072
4073
4074
4075
4076
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4089
4090
4091
4092
4095
4096
4097
4098
4100
4101
4102
4103
4104
4105
4106
4107
4109
4111
4113
4114
4116
4117
4121
4122
4123
4126
4129
4132
4133
4134
4135
4136
4137
4138
4140
4142
4143
4145
4149
4151
4152
4156
4158
4160
4162
4163
4164
4165
4166
4167
4168
4170
4171
4172
4173
4175
4176
4178
4179
4181
4183
4184
4187
4189
4190
4191
4192
4193
4194
4195
4196
4197
4198
4199
4200
4201
4202
4203
4204
4205
4207
4209
4212
4213
4214
4215
4217
4219
4220
4222
4223
4224
4226
4228
4230
4231
4232
4233
4235
4237
4238
4239
4240
4241
4242
4244
4245
4248
4249
4250
4251
4253
4255
4257
4258
4259
4260
4262
4263
4264
4266
4267
4268
4269
4270
4271
4272
4273
4274
4275
4276
4278


In [29]:
json_object = json.dumps(data_dict, indent=4, ensure_ascii=False)
with open("/content/drive/MyDrive/Startup_Scraping/startup_list_0_5000_without_cookie.json", "w", encoding='utf-8') as outfile:
    outfile.write(json_object)